<a href="https://colab.research.google.com/github/Carolinsrainbow/UC_IngenieriaDeDatos/blob/main/Ayudantia2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [61.3 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,176 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/un

# **HAVING**

Datos de ejemplo (adaptado de https://www.w3schools.com/sql/sql_having.asp)

In [ ]:
%%sql

DROP TABLE IF EXISTS Customers;

CREATE TABLE IF NOT EXISTS Customers(
  cid int PRIMARY KEY,
  customerName VARCHAR(50),
  contactName VARCHAR(50),
  address VARCHAR(50),
  city VARCHAR(50),
  postalCode VARCHAR(50),
  country VARCHAR(50)
);

INSERT INTO Customers VALUES(1, 'Juan', 'Juan', 'Calle 1', 'Bogota', '11111', 'Colombia');
INSERT INTO Customers VALUES(3, 'Pedro', 'Pedro', 'Calle 2', 'Lima', '11111', 'Peru');
INSERT INTO Customers VALUES(5, 'Diego', 'Diego', 'Calle 3', 'Buenos Aires', '11111', 'Argentina');
INSERT INTO Customers VALUES(6, 'Pablo', 'Pablo', 'Calle 4', 'Santiago', '11111', 'Chile');
INSERT INTO Customers VALUES(7, 'Vale', 'Vale', 'Calle 5', 'Santiago', '11111', 'Chile');
INSERT INTO Customers VALUES(10, 'Dafne', 'Dafbe', 'Calle 6', 'Santiago', '11111', 'Chile');

 * postgresql+psycopg2://@/postgres


""


Queremos contar cuantos clientes hay por pais, pero solo en aquellos donde hay más de 2 clientes.

In [ ]:
%%sql

SELECT COUNT(cid), country
FROM Customers
GROUP BY country
WHERE COUNT(cid) > 2;

 * postgresql+psycopg2://@/postgres
(psycopg2.errors.SyntaxError) syntax error at or near "WHERE"
LINE 4: WHERE COUNT(cid) > 2;
        ^

[SQL: SELECT COUNT(cid), country
FROM Customers
GROUP BY country
WHERE COUNT(cid) > 2;]
(Background on this error at: https://sqlalche.me/e/20/f405)


Usando HAVING

In [ ]:
%%sql

SELECT COUNT(cid), country
FROM Customers
GROUP BY country
HAVING COUNT(cid) > 2;

 * postgresql+psycopg2://@/postgres


,count,country
0,3,Chile


Consulta equivalente, usen lo que les resulte más natural (considerando limitaciones de su consulta particular)

In [ ]:
%%sql
SELECT idCount, country
FROM (
	SELECT COUNT(cid) as idCount, country
  FROM Customers
  GROUP BY country
) as countTable
WHERE idCount > 2

 * postgresql+psycopg2://@/postgres


,idcount,country
0,3,Chile


# **SURROGATE KEYS: INT VS SERIAL** (Material de repaso en casa)


In [ ]:
%%sql
DROP TABLE IF EXISTS Tiendas;
CREATE TABLE Tiendas(tid INT PRIMARY KEY, capacidad INT, minpersonal INT);
INSERT INTO Tiendas VALUES(1, 100, 10);
INSERT INTO Tiendas VALUES(2, 150, 15);
INSERT INTO Tiendas VALUES(3, 50, 5);

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%%sql
SELECT * FROM Tiendas;

 * postgresql+psycopg2://@/postgres


,tid,capacidad,minpersonal
0,1,100,10
1,2,150,15
2,3,50,5


Noten que con SERIAL podemos olvidarnos del id, solo usamos default y agrega el id en orden.

In [ ]:
%%sql
DROP TABLE IF EXISTS Tiendas;
CREATE TABLE Tiendas(tid SERIAL PRIMARY KEY, capacidad INT, minpersonal INT);
INSERT INTO Tiendas VALUES(default, 100, 10);
INSERT INTO Tiendas VALUES(default, 150, 15);
INSERT INTO Tiendas VALUES(default, 50, 5);

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%%sql
SELECT * FROM Tiendas;

 * postgresql+psycopg2://@/postgres


,tid,capacidad,minpersonal
0,1,100,10
1,2,150,15
2,3,50,5


# **CONSULTAS ANIDADAS**

Datos

In [ ]:
%%sql
DROP TABLE IF EXISTS Tiendas;
CREATE TABLE Tiendas(tid INT PRIMARY KEY, capacidad INT, minpersonal INT);
INSERT INTO Tiendas VALUES(1, 100, 10);
INSERT INTO Tiendas VALUES(2, 150, 15);
INSERT INTO Tiendas VALUES(3, 50, 5);

DROP TABLE IF EXISTS Productos;
CREATE TABLE Productos(pid INT PRIMARY KEY, nombre VARCHAR(200), precio FLOAT, tipo VARCHAR(100));
INSERT INTO Productos VALUES(100, 'Polera BTS', 40, 'Ropa');
INSERT INTO Productos VALUES(200, 'Polera Bad Bunny', 20, 'Ropa');
INSERT INTO Productos VALUES(300, 'Refrigerador', 1000, 'Electro');
INSERT INTO Productos VALUES(400, 'Mancuerna 10kgs', 80, 'Deporte');
INSERT INTO Productos VALUES(500, 'Banda elástica', 10, 'Deporte');
INSERT INTO Productos VALUES(600, 'Pelota Futbol', 200, 'Deporte');
INSERT INTO Productos VALUES(700, 'Cafetera', 100, 'Electro');
INSERT INTO Productos VALUES(800, 'Monitor', 200, 'Tecnología');
INSERT INTO Productos VALUES(900, 'Notebook', 1200, 'Tecnología');
INSERT INTO Productos VALUES(1000, 'Lomo kg', 10, 'Carne');

DROP TABLE IF EXISTS Vende;
CREATE TABLE Vende(tid INT, pid INT, PRIMARY KEY(tid, pid),
                    FOREIGN KEY (tid) REFERENCES Tiendas(tid), FOREIGN KEY (pid) REFERENCES Productos(pid));
INSERT INTO Vende VALUES(1, 100);
INSERT INTO Vende VALUES(1, 200);
INSERT INTO Vende VALUES(1, 300);
INSERT INTO Vende VALUES(1, 400);
INSERT INTO Vende VALUES(1, 500);
INSERT INTO Vende VALUES(1, 600);
INSERT INTO Vende VALUES(1, 700);
INSERT INTO Vende VALUES(1, 800);
INSERT INTO Vende VALUES(1, 900);
INSERT INTO Vende VALUES(2, 100);
INSERT INTO Vende VALUES(2, 200);
INSERT INTO Vende VALUES(2, 300);
INSERT INTO Vende VALUES(2, 700);
INSERT INTO Vende VALUES(3, 100);
INSERT INTO Vende VALUES(3, 200);
INSERT INTO Vende VALUES(3, 300);
INSERT INTO Vende VALUES(3, 400);
INSERT INTO Vende VALUES(3, 500);
INSERT INTO Vende VALUES(3, 700);
INSERT INTO Vende VALUES(3, 800);
INSERT INTO Vende VALUES(3, 900);
INSERT INTO Vende VALUES(1, 1000);

DROP TABLE IF EXISTS Usuarios;
CREATE TABLE Usuarios(uid INT PRIMARY KEY, nombre VARCHAR(200), calle VARCHAR(200), comuna VARCHAR(100), region VARCHAR(200));
INSERT INTO Usuarios VALUES(1, 'Valentina', 'Calle 123', 'Conchalí', 'RM');
INSERT INTO Usuarios VALUES(2, 'Adrián', 'Dirección 145', 'Maipú', 'RM');
INSERT INTO Usuarios VALUES(5, 'Lomos', 'Dirección 90', 'Maipú', 'RM');
INSERT INTO Usuarios VALUES(3, 'Conito', 'Otra Calle 78', 'Viña del mar', 'Valparaíso');
INSERT INTO Usuarios VALUES(4, 'Frambuesa', 'Más direcciones 90', 'Concón', 'Valparaíso');

DROP TABLE IF EXISTS Compras;
CREATE TABLE Compras(cid INT PRIMARY KEY, uid INT, tid INT);
INSERT INTO Compras VALUES(1, 1, 1);
INSERT INTO Compras VALUES(101, 1, 1);
INSERT INTO Compras VALUES(3, 1, 1);
INSERT INTO Compras VALUES(4, 1, 2);
INSERT INTO Compras VALUES(5, 1, 2);
INSERT INTO Compras VALUES(6, 2, 1);
INSERT INTO Compras VALUES(7, 2, 2);
INSERT INTO Compras VALUES(8, 2, 3);
INSERT INTO Compras VALUES(9, 3, 3);
INSERT INTO Compras VALUES(10, 3, 3);
INSERT INTO Compras VALUES(11, 3, 1);
INSERT INTO Compras VALUES(12, 4, 1);

DROP TABLE IF EXISTS Contenido_Compra;
CREATE TABLE Contenido_Compra(cid INT, pid INT, cantidad INT, PRIMARY KEY(cid, pid),
                              FOREIGN KEY (cid) REFERENCES Compras(cid), FOREIGN KEY (pid) REFERENCES Productos(pid));
INSERT INTO Contenido_Compra VALUES(1, 100, 2);
INSERT INTO Contenido_Compra VALUES(101, 800, 2);
INSERT INTO Contenido_Compra VALUES(101, 900, 1);
INSERT INTO Contenido_Compra VALUES(101, 100, 1);
INSERT INTO Contenido_Compra VALUES(101, 1000, 3);
INSERT INTO Contenido_Compra VALUES(3, 400, 2);
INSERT INTO Contenido_Compra VALUES(4, 200, 3);
INSERT INTO Contenido_Compra VALUES(5, 100, 1);
INSERT INTO Contenido_Compra VALUES(6, 100, 3);
INSERT INTO Contenido_Compra VALUES(6, 600, 1);
INSERT INTO Contenido_Compra VALUES(7, 300, 1);
INSERT INTO Contenido_Compra VALUES(7, 700, 1);
INSERT INTO Contenido_Compra VALUES(8, 200, 4);
INSERT INTO Contenido_Compra VALUES(8, 1000, 2);
INSERT INTO Contenido_Compra VALUES(9, 100, 2);
INSERT INTO Contenido_Compra VALUES(10, 100, 1);
INSERT INTO Contenido_Compra VALUES(11, 200, 1);
INSERT INTO Contenido_Compra VALUES(12, 500, 3);

 * postgresql+psycopg2://@/postgres


""


7. Nombre de los usuarios que compraron uno de los cinco productos más caros. Para sacar los cinco más caros, usar LIMIT:

In [ ]:
%%sql
SELECT DISTINCT Usuarios.nombre
FROM Usuarios, Compras, Contenido_Compra
WHERE Compras.cid = Contenido_Compra.cid
AND Compras.uid = Usuarios.uid
AND pid IN (SELECT pid FROM Productos ORDER BY precio DESC LIMIT 5);

 * postgresql+psycopg2://@/postgres


,nombre
0,Adrián
1,Valentina


Notemos que la consulta anidada también funciona por si sola

In [ ]:
%%sql
SELECT pid FROM Productos
ORDER BY precio DESC
LIMIT 5

 * postgresql+psycopg2://@/postgres


,pid
0,900
1,300
2,800
3,600
4,700


Otra cosa interesante a notar es que utlizando la palabra reservada UNION podemos unir los resultados de consultas en una tabla. Por ejemplo: Buscar los nombres de los usuarios que compraron uno de los cinco productos más caros y/o de los cinco más baratos.

In [ ]:
%%sql
SELECT DISTINCT Usuarios.nombre
FROM (
  SELECT DISTINCT Usuarios.nombre
  FROM Usuarios, Compras, Contenido_Compra
  WHERE Compras.cid = Contenido_Compra.cid
  AND Compras.uid = Usuarios.uid
  AND pid IN (SELECT pid FROM Productos ORDER BY precio DESC LIMIT 5)
  UNION
  SELECT DISTINCT Usuarios.nombre
  FROM Usuarios, Compras, Contenido_Compra
  WHERE Compras.cid = Contenido_Compra.cid
  AND Compras.uid = Usuarios.uid
  AND pid IN (SELECT pid FROM Productos ORDER BY precio ASC LIMIT 5)
) as Usuarios;


 * postgresql+psycopg2://@/postgres


,nombre
0,Adrián
1,Conito
2,Frambuesa
3,Valentina


8. Id de cada tienda, junto con el promedio de los precios de todos los productos que vende esta tienda

In [ ]:
%%sql
SELECT tid As id_tienda, AVG(Productos.precio) AS precio_promedio
FROM Vende, Productos
WHERE Vende.pid = Productos.pid
GROUP BY tid;

 * postgresql+psycopg2://@/postgres


,id_tienda,precio_promedio
0,3,331.25
1,2,290.00
2,1,286.00


9. Id de todas las compras en las cuales se compraron más de tres productos distintos (si el mismo producto fue comprado 3 veces o mas, esto no cuenta)

In [ ]:
%%sql
SELECT Compras.cid
FROM Compras, (
      SELECT cid, COUNT(DISTINCT pid) AS cantidad
      FROM Contenido_Compra
      GROUP BY cid
      ) As Productos_distintos
WHERE Productos_distintos.cantidad >= 3
AND Productos_distintos.cid = Compras.cid;

 * postgresql+psycopg2://@/postgres


,cid
0,101


10. Para cada compra con **tres o más productos**, listar su id, el nombre del usuario, y el número de productos comprados en esta compra. Si un producto fue comprado dos o más veces, hay que contar esto:

In [ ]:
%%sql
SELECT Compras.cid, Usuarios.nombre, ProductosEnCompra.total
FROM Compras, Usuarios,
      (
        SELECT cid, SUM(cantidad) AS total
        FROM Contenido_Compra
        GROUP BY cid
      ) AS ProductosEnCompra
WHERE ProductosEnCompra.total >= 3
AND ProductosEnCompra.cid = Compras.cid
AND Usuarios.uid = Compras.uid;

 * postgresql+psycopg2://@/postgres


,cid,nombre,total
0,101,Valentina,7
1,4,Valentina,3
2,6,Adrián,4
3,8,Adrián,6
4,12,Frambuesa,3


11. Id de todas las tiendas que venden al menos un producto más caro que el producto más barato que existe en todos los productos:

In [ ]:
%%sql
SELECT DISTINCT tid
FROM Vende, Productos
WHERE Vende.pid = Productos.pid
AND Productos.precio > ANY (SELECT MIN(precio) FROM Productos);

 * postgresql+psycopg2://@/postgres


,tid
0,3
1,2
2,1


In [ ]:
%%sql
SELECT DISTINCT tid
FROM Vende, Productos
WHERE Vende.pid = Productos.pid
AND Productos.precio > (SELECT MIN(precio) FROM Productos);

 * postgresql+psycopg2://@/postgres


,tid
0,3
1,2
2,1


12. Para cada región, listar el id de la tienda más preferida por los usuarios en esta región. Aquí se cuenta el número total de compras realizadas en la tienda, y no el número de usuarios que realizaron la compra. Si dos o mas tiendas empatadas para el primer lugar, listar todas.

In [ ]:
%%sql
WITH
ComprasXTiendaXRegion As
      (SELECT region, tid, COUNT(cid) As total_compras
      FROM Usuarios, Compras
      WHERE Usuarios.uid = Compras.uid
      GROUP BY region, tid),

MaxComprasRegion As
      (SELECT region, MAX(total_compras) As max
      FROM ComprasXTiendaXRegion
      GROUP BY region)

SELECT ctr.region, ctr.tid
FROM ComprasXTiendaXRegion As ctr, MaxComprasRegion
WHERE ctr.total_compras = MaxComprasRegion.max
AND ctr.region = MaxComprasRegion.region;

 * postgresql+psycopg2://@/postgres


,region,tid
0,RM,1
1,Valparaíso,3
2,Valparaíso,1
